In [ ]:
#| default_exp helpers.submit

# Notebook Submission Helper (submit.py)

> This helper supports submitting completed notebooks for credit by extracting student responses and sending them to a remote grading service.

Answer format convention

Student answers are expected to appear in code cells using the following pattern:
```python
q1_answer = "A"
q2_answer = 3.14
q3_answer = ["token", "vector", "embedding"]
```
Where:
- Each answer variable name begins with q
- The question number follows (q1, q2, …)
- The variable is assigned a valid Python literal

This convention allows answers to be reliably extracted from the notebook without requiring special widgets or forms.

Submission workflow

The submission process has three steps:

1. Collect answers
Code cells matching the answer pattern are located and extracted.

2. Parse answers
Extracted code is converted into a structured Python dictionary.

3. Submit answers
The parsed answers are sent via HTTP to a grading service.

Each step is implemented as a separate function to keep the logic clear and testable.

In [ ]:
#| export
import os
import httpx
import nbformat
from pathlib import Path
import inspect
import re
import json

In [ ]:
#| export
def _can_use_dialoghelper():
    try:
        from dialoghelper import find_msgs
        return True
    except Exception:
        return False

In [ ]:
#| export
def _collect_answers_from_notebook(path):
    """
    Collect qN_answer assignments by parsing a notebook file directly.

    NOTE: In Jupyter, __file__ is usually not defined, so we REQUIRE `path`.
    """
    if path is None:
        raise RuntimeError(
            "Notebook path is required in Jupyter.\n"
            "Call collect_answers(..., path='YOUR_NOTEBOOK.ipynb')"
        )

    nb = nbformat.read(Path(path), as_version=4)

    results = []
    pat = re.compile(r'^q\d+_answer\s*=', re.MULTILINE)

    for cell in nb.cells:
        if cell.cell_type == "code":
            for line in cell.source.splitlines():
                if pat.match(line.strip()):
                    results.append(("code", line.strip()))

    return results



In [ ]:
#| export
def collect_answers(show=True, *, namespace=None, dname=None, path=None):
    """
    Collect student answers.

    Priority:
    1. Live Python variable state (default, works everywhere)
    2. dialoghelper (optional, Solveit)
    3. Notebook file parsing (local Jupyter only, requires path)

    Parameters
    ----------
    show : bool
        Print extracted answers
    namespace : dict, optional
        Namespace to search (defaults to globals())
    dname : str, optional
        Dialog name (dialoghelper only)
    path : str or Path, optional
        Notebook path (local fallback only)

    Returns
    -------
    list[tuple]
        [('code', 'q1_answer = ...'), ...]
    """

    # --- 1. Variable-state collection (canonical) ---
    if namespace is None:
        namespace = globals()

    pattern = re.compile(r"^q\d+_answer$")
    results = []

    for name, value in namespace.items():
        if pattern.match(name):
            results.append(("code", f"{name} = {repr(value)}"))

    if results:
        if show:
            print("=== Student Responses (from variable state) ===\n")
            for _, answer in results:
                print(answer)
        return results

    # --- 2. dialoghelper fallback (optional) ---
    if dname is not None:
        try:
            from dialoghelper import find_msgs
            msgs = find_msgs(
                re_pattern=r'^q\d+_answer\s*=',
                msg_type='code',
                dname=dname
            )
            results = [('code', msg['content'].strip()) for msg in msgs]
            if results:
                return results
        except Exception:
            pass

    # --- 3. Notebook parsing fallback (local only) ---
    if path is not None:
        return _collect_answers_from_notebook(path)

    raise RuntimeError(
        "No answers found.\n"
        "Expected variables named q*_answer in the notebook."
    )



In [ ]:
#| export
def parse_answers(raw):
    """
    Parse extracted answer code into a dictionary.

    Parameters
    ----------
    raw : list of tuple
        Output from `collect_answers`, containing raw code strings.

    Returns
    -------
    dict
        Mapping from answer variable name (e.g. 'q1_answer') to its value.
    """
    result = {}
    for typ, content in raw:
        if typ == 'code':
            # Parse "q1_answer = "A"" format
            key, val = content.split(' = ', 1)
            result[key] = eval(val)
    return result

In [ ]:
#| export
def submit_answers(
    student_id,
    *,
    answers=None,
    raw_answers=None,
    dname=None,
    path=None,
    url="https://nbsubmit-production.up.railway.app",
    api_key=None,
    verbose=True,
):
    if not student_id.strip():
        raise ValueError("student_id cannot be empty")

    # --- Canonical resolution of answers (ONCE) ---
    if answers is None:
        if raw_answers is None:
            raw_answers = collect_answers(
                show=False,
                dname=dname,
                path=path
            )
        answers = parse_answers(raw_answers)

    if not answers:
        raise RuntimeError("No answers found to submit.")

    payload = {
        "student_id": student_id,
        "answers": answers,
    }

    if verbose:
        print("\n📤 YOUR SUBMISSION")
        print("=" * 60)
        print(json.dumps(payload, indent=2))
        print("=" * 60)

    key = api_key or os.environ.get("SUBMIT_API_KEY")
    if not key:
        raise RuntimeError(
            "Missing SUBMIT_API_KEY. Set it in your environment or .env file."
        )

    headers = {"x-api-key": key}
    response = httpx.post(
        f"{url}/submit",
        json=payload,
        headers=headers
    )
    return response.json()


In [ ]:
# Test (not exported)

from data401_nlp.helpers.submit import parse_answers

In [ ]:
#| export
def review_answers(*, namespace=None, dname=None, path=None, show=True):
    """
    Review extracted student answers WITHOUT submitting.

    Priority:
    1. Live variable state
    2. dialoghelper (optional)
    3. Notebook parsing (local only, requires path)

    Parameters
    ----------
    namespace : dict, optional
        Namespace to search (defaults to globals())
    dname : str, optional
        Dialog name (Solveit / Deepnote)
    path : str or Path, optional
        Notebook path (local fallback only)
    show : bool
        Print formatted output.

    Returns
    -------
    dict
        Parsed answers ready for submission.
    """

    raw = collect_answers(
        show=False,
        namespace=namespace,
        dname=dname,
        path=path
    )

    answers = parse_answers(raw)

    if show:
        print("\n📝 REVIEW: Answers detected")
        print("=" * 60)
        for k, v in answers.items():
            print(f"{k}: {v!r}")
        print("=" * 60)
        print("⚠️  Nothing has been submitted.")

    return answers


In [ ]:
raw = [
    ("code", 'q1_answer = "A"'),
    ("code", "q2_answer = 42"),
]

assert parse_answers(raw) == {
    "q1_answer": "A",
    "q2_answer": 42,
}

print("parse_answers OK")


parse_answers OK
